In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from pyproj import Proj
from scipy import interpolate

myProj = Proj("+proj=utm +zone=54 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

In [ ]:
df = pd.read_csv('/Users/olangoea/Downloads/PNGMainShock.fsp', skiprows=51, delim_whitespace=True, 
                 names = ['lat','lon','x','y','z','slip','rake','rupt_time','rise','moment'])

df

In [ ]:
#Summing moments
rt=np.sum(df['moment'])
rt

In [ ]:
#define strike and dip from file
strike,dip = 308, 33

#convert coordinates to utm for interpolation
utmx, utmy = myProj(df.lon, df.lat)
centroid = (sum(utmx) / len(df), sum(utmy) / len(df))

#rotate the fault model about the central point so interpolation is easy
xshift, yshift = utmx-centroid[0], utmy-centroid[1]
xp = xshift*np.cos(strike*np.pi/180) - yshift*np.sin(strike*np.pi/180) + centroid[0]
yp = xshift*np.sin(strike*np.pi/180) + yshift*np.cos(strike*np.pi/180) + centroid[1]

In [ ]:
#make an interpolation surface
points = np.hstack((xp.reshape(len(xp),1), yp.reshape(len(xp),1)))
fz = interpolate.LinearNDInterpolator(points, df.z, fill_value='nan')
frise = interpolate.LinearNDInterpolator(points, df.rise, fill_value='nan')
frupt = interpolate.LinearNDInterpolator(points, df.rupt_time, fill_value='nan')
fmo = interpolate.LinearNDInterpolator(points, df.moment, fill_value='nan')
frake = interpolate.LinearNDInterpolator(points, df.rake, fill_value='nan')
fslip = interpolate.LinearNDInterpolator(points, df.slip, fill_value='nan')

In [ ]:
fslip

In [ ]:
xmin,xmax = np.min(xp),np.max(xp)
ymin,ymax = np.min(yp),np.max(yp)

xspace,yspace = 500,500

xrange = np.arange(xmin,xmax,xspace)
yrange = np.arange(ymin,ymax,yspace)

X,Y = np.meshgrid(xrange,yrange)

z = fz(X,Y)
rise = frise(X,Y)
rupt = frupt(X,Y)
mo = fmo(X,Y)
rake = frake(X,Y)
slip = fslip(X,Y)

In [ ]:
np.sum(mo)

In [ ]:
(slip.ravel()).shape

In [ ]:
#put everything together as xy points
stack = (X.ravel(), Y.ravel(), z.ravel(), rupt.ravel(), rise.ravel(), rake.ravel(), mo.ravel(), slip.ravel())


x = stack[0]
y = stack[1]
#rotate back
xshift, yshift = x-centroid[0], y-centroid[1]
xp = xshift*np.cos(-1*strike*np.pi/180) - yshift*np.sin(-1*strike*np.pi/180) + centroid[0]
yp = xshift*np.sin(-1*strike*np.pi/180) + yshift*np.cos(-1*strike*np.pi/180) + centroid[1]

In [ ]:
fig,axs = plt.subplots(3,2, figsize = (20,10))

im1=axs[0,0].scatter(xp,yp,c = z, label = 'Depth')
fig.colorbar(im1, ax=axs[0, 0])

im2=axs[0,1].scatter(xp,yp,c = rupt, label = 'rupture time')
fig.colorbar(im2, ax=axs[0, 1])
im3=axs[1,0].scatter(xp,yp,c = stack[4], label = 'rise time')
fig.colorbar(im3, ax=axs[1, 0])
im4=axs[1,1].scatter(xp,yp,c = stack[5], label = 'rake')
fig.colorbar(im4, ax=axs[1, 1])
im5=axs[2,0].scatter(xp,yp,c = stack[6], label = 'moment')
fig.colorbar(im5, ax=axs[2, 0])
im6=axs[2,1].scatter(xp,yp,c = stack[7], label = 'slip')
fig.colorbar(im6, ax=axs[2, 1])
#plt.show()
plt.legend()
#plt.scatter(xp,yp,c = stack[3], s = 0.1)
#plt.colorbar()
plt.show()

In [ ]:
#sum up values of moments
moment1 = np.sum(stack[6])
moment2=np.sum(mo)
print(moment1, moment2)

In [ ]:
fig,axs = plt.subplots(3,2, figsize = (20,10))

im1=axs[0,0].scatter(xp,yp,c = stack[2], label = 'Depth')
fig.colorbar(im1, ax=axs[0, 0])

im2=axs[0,1].scatter(xp,yp,c = stack[3], label = 'rupture time')
fig.colorbar(im2, ax=axs[0, 1])
im3=axs[1,0].scatter(xp,yp,c = stack[4], label = 'rise time')
fig.colorbar(im3, ax=axs[1, 0])
im4=axs[1,1].scatter(xp,yp,c = stack[5], label = 'rake')
fig.colorbar(im4, ax=axs[1, 1])
im5=axs[2,0].scatter(xp,yp,c = stack[6], label = 'moment')
fig.colorbar(im5, ax=axs[2, 0])
im6=axs[2,1].scatter(xp,yp,c = stack[7], label = 'slip')
fig.colorbar(im6, ax=axs[2, 1])
#plt.show()
plt.legend()
#plt.scatter(xp,yp,c = stack[3], s = 0.1)
#plt.colorbar()
plt.show()

In [ ]:
plt.scatter(xp,yp,c = stack[3], label = 'rupture time',s=0.1)
plt.colorbar()
plt.show()

In [ ]:
#calculate the area of one subfault using the strike and dip
area = xspace/np.cos(dip * np.pi/180) * yspace/np.cos(strike-360 * np.pi/180)
print('Area for 1 subfault = ' + str(round(area/1e6,5)) + ' km2')

#Then use the interpolated slip to calculate a new moment, it was confusing the other way around....
mo = stack[7]*area*30e9

In [ ]:
moment = mo[~np.isnan(mo)]
# moment.flatten()

In [ ]:
#compare to make sure new and old moment match up
plt.scatter(xp,yp,c=mo)
plt.colorbar()
plt.show()

plt.scatter(xp,yp,c=stack[6],cmap='turbo')
plt.colorbar()
plt.show()



In [ ]:
#compare to make sure new and old moment match up
plt.scatter(xp,yp,c=slip)
plt.colorbar()
plt.show()

plt.scatter(xp,yp,c=stack[7],cmap='turbo')
plt.colorbar()
plt.show()

In [ ]:
from pyrocko import moment_tensor as pmt

In [ ]:
strk=strike*np.ones_like(moment)
dp=dip*np.ones_like(moment)


In [ ]:
dt=pd.DataFrame(stack[6])
dt['rake']=stack[5]
dt['x']=xp
dt['y']=yp
dt['z']=stack[2]
dt['rupture']=stack[3]
dt['rise']=stack[4]
dt['slip']=stack[7]
dt.columns=['moment','rake','x','y','z','rupture','rise','slip']
dt= dt.dropna()
dt

In [ ]:
# dt1=pd.DataFrame(stack[6])
# dt['rake']=stack[5]
# dt['x']=xp
# dt['y']=yp
# dt['z']=stack[2]
# dt['rupture']=stack[3]
# dt['rise']=stack[4]
# dt['slip']=stack[7]
# dt.columns=['moment','rake','x','y','z','rupture','rise','slip']
# dt= dt.dropna()
# dt

In [ ]:
# dt=pd.DataFrame(stack[6])
# dt['rake']=stack[5]
# dt['x']=xp
# dt['y']=yp
# dt['z']=z.flatten()
# dt['rupture']=rupt.flatten()
# dt['rise']=rise.flatten()
# dt['slip']=slip.flatten()
# dt.columns=['moment','rake','x','y','z','rupture','rise','slip']
# dt= dt.dropna()
# dt
# # z = fz(X,Y)
# # rise = frise(X,Y)
# # rupt = frupt(X,Y)
# # mo = fmo(X,Y)
# # rake = frake(X,Y)
# # slip = fslip(X,Y)

In [ ]:
dt.to_csv("/Users/olangoea/Library/CloudStorage/OneDrive-KAUST/Documents/day-03-material/interpolated_flatten.csv")

In [ ]:
plt.scatter(dt['x'],dt['y'],c=dt['rupture'])
plt.colorbar()
plt.show()

In [ ]:
mrr=[]
mtt=[]
mpp=[]
mrt=[]
mrp=[]
mtp=[]
for idx,row in dt.iterrows():
    rake= row.rake
    moment= row.moment
    mt = pmt.MomentTensor(strike=strike, dip=dip, rake=rake, scalar_moment=moment)
    # m.append(mt.mnn, mt.mee, mt.mdd, mt.mne, mt.mnd, mt.med)  # The six MT components
    mrr.append(mt.mnn)
    mtt.append(mt.mee)
    mpp.append(mt.mdd) 
    mrt.append(mt.mne) 
    mrp.append(mt.mnd) 
    mtp.append(mt.med)

In [ ]:
dt1 = dt.iloc[:,[2,3,4,5,6]] # Select columns by Index

new_dataframe = dt1.assign(Mrr = mrr,Mtt=mtt,Mpp=mpp,Mrt=mrt,Mrp=mrp,Mtp=mtp)
new_dataframe

In [ ]:
dt2 = dt.copy() # Select columns by Index

new_dataframe = dt2.assign(Mrr = mrr,Mtt=mtt,Mpp=mpp,Mrt=mrt,Mrp=mrp,Mtp=mtp)
new_dataframe= new_dataframe.drop(columns=['x','y','lat','lon'])
new_dataframe['x']=utmx
new_dataframe['y']=utmy
new_dataframe

In [ ]:
new_dataframe.to_csv("old_fault.csv")